In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/vk-intern/intern_task.csv


In [2]:
df = pd.read_csv('/kaggle/input/vk-intern/intern_task.csv')

In [3]:
df.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000


1 ШАГ: Работа с данными. 

1) Проверяем, есть ли в нашем датафрейме пропуски 


In [6]:
missing_values = df.isna().any()

In [7]:
missing_values[missing_values == True]

Series([], dtype: bool)

Получили пустой df, значит, во всех ячейках есть элементы

2) Проверим, есть ли в нашем датафрейме что-то, кроме int и float64, чтобы понять, нужно ли заменять текстовые данные

In [8]:
df_dtypes = df.dtypes

In [9]:
df_dtypes[df_dtypes != 'float64']

rank           int64
query_id       int64
feature_64     int64
feature_65     int64
feature_72     int64
feature_100    int64
dtype: object

Видим, что весь наш датасет состоит только из чисел

2 ШАГ: Обучение модели

Обучим нашу модель с помощью библиотеки catboost, основанной на градиентном бустинге.
Градиентный бустинг я выбрал, потому что мы работаем с табличными данными, поэтому бустинг будет хорошо решать эту задачу

In [11]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

Попробуем обучить нашу модель только на первой сессии

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[df['query_id'] == 10].drop('rank', axis=1), df[df['query_id'] == 10]['rank'], test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=150, learning_rate=0.1, depth=6, loss_function='MultiClass')

In [13]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Вычисление NDCG
b = np.reshape(predictions, y_test.shape)
ndcg = ndcg_score([y_test], [b])
print("NDCG:", ndcg)

0:	learn: 1.5438826	total: 68ms	remaining: 10.1s
1:	learn: 1.4935862	total: 75.8ms	remaining: 5.61s
2:	learn: 1.4308371	total: 82.6ms	remaining: 4.05s
3:	learn: 1.3794257	total: 89.2ms	remaining: 3.26s
4:	learn: 1.3350271	total: 93.2ms	remaining: 2.7s
5:	learn: 1.3011045	total: 99.6ms	remaining: 2.39s
6:	learn: 1.2702084	total: 107ms	remaining: 2.18s
7:	learn: 1.2269533	total: 113ms	remaining: 2.01s
8:	learn: 1.1905616	total: 120ms	remaining: 1.88s
9:	learn: 1.1581760	total: 124ms	remaining: 1.74s
10:	learn: 1.1286205	total: 128ms	remaining: 1.62s
11:	learn: 1.1046696	total: 136ms	remaining: 1.56s
12:	learn: 1.0836861	total: 143ms	remaining: 1.51s
13:	learn: 1.0585665	total: 150ms	remaining: 1.46s
14:	learn: 1.0293350	total: 157ms	remaining: 1.42s
15:	learn: 1.0038677	total: 165ms	remaining: 1.38s
16:	learn: 0.9875994	total: 175ms	remaining: 1.37s
17:	learn: 0.9689606	total: 183ms	remaining: 1.34s
18:	learn: 0.9524677	total: 191ms	remaining: 1.31s
19:	learn: 0.9317680	total: 198ms	rema

Получили nDCG в районе 0.9, что показывает, что модель хорошо справляется с задачей

Теперь обучим нашу модель на всём датасете:

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('rank', axis=1), df['rank'], test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=150, learning_rate=0.2, depth=6, loss_function='MultiClass')

In [19]:
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Вычисление NDCG
b = np.reshape(predictions, y_test.shape)
ndcg = ndcg_score([y_test], [b])
print("NDCG:", ndcg)

0:	learn: 1.4094530	total: 539ms	remaining: 1m 20s
1:	learn: 1.2962729	total: 1.09s	remaining: 1m 21s
2:	learn: 1.2189129	total: 1.69s	remaining: 1m 22s
3:	learn: 1.1656846	total: 2.27s	remaining: 1m 22s
4:	learn: 1.1261428	total: 2.89s	remaining: 1m 23s
5:	learn: 1.0958271	total: 3.29s	remaining: 1m 18s
6:	learn: 1.0733216	total: 3.8s	remaining: 1m 17s
7:	learn: 1.0549286	total: 4.29s	remaining: 1m 16s
8:	learn: 1.0408459	total: 4.86s	remaining: 1m 16s
9:	learn: 1.0308602	total: 5.37s	remaining: 1m 15s
10:	learn: 1.0230913	total: 5.97s	remaining: 1m 15s
11:	learn: 1.0151774	total: 6.44s	remaining: 1m 14s
12:	learn: 1.0100583	total: 6.75s	remaining: 1m 11s
13:	learn: 1.0056177	total: 7.12s	remaining: 1m 9s
14:	learn: 1.0019960	total: 7.57s	remaining: 1m 8s
15:	learn: 0.9995447	total: 7.95s	remaining: 1m 6s
16:	learn: 0.9969553	total: 8.36s	remaining: 1m 5s
17:	learn: 0.9949730	total: 8.8s	remaining: 1m 4s
18:	learn: 0.9927251	total: 9.18s	remaining: 1m 3s
19:	learn: 0.9903336	total: 9.

Получили nDCG равный 0.915